# Asyncio并发编程教程

本教程将通过实际可运行的代码示例，系统地介绍Python中的asyncio并发编程。包括基础概念、核心组件、异步操作、高级特性以及最佳实践等内容。

## 1. 基础概念

### 1.1 事件循环(Event Loop)
事件循环是asyncio的核心，负责协调和调度所有异步任务。主要组成：
- 回调系统
- 驱动生成器
- IO多路复用(epoll)

让我们通过一个简单的例子来了解事件循环的基本使用：

In [ ]:
import asyncio
import time

async def hello(name):
    print(f'{name}: 开始')
    await asyncio.sleep(1)  # 模拟IO操作
    print(f'{name}: 结束')

async def main():
    # 创建多个协程任务
    await asyncio.gather(
        hello('任务1'),
        hello('任务2'),
        hello('任务3')
    )

# 运行事件循环
if __name__ == '__main__':
    start = time.time()
    asyncio.run(main())
    print(f'总用时: {time.time() - start:.2f}秒')

### 1.2 协程(Coroutine)
协程是可以暂停和恢复执行的函数，使用async/await语法定义。主要特点：
- 单线程内实现并发
- 显式切换点
- 避免回调地狱

下面是一个展示协程基本用法的例子：

In [ ]:
async def fetch_data():
    print('开始获取数据...')
    await asyncio.sleep(2)  # 模拟IO操作
    return {'data': 'some data'}

async def process_data():
    print('等待数据...')
    data = await fetch_data()  # 等待数据获取完成
    print('数据处理完成:', data)

# 运行示例
if __name__ == '__main__':
    asyncio.run(process_data())

## 2. 核心组件

### 2.1 Future对象
Future对象表示一个异步操作的最终结果。它提供了一种方式来查看操作是否完成、获取结果或异常等。

In [ ]:
async def set_future_result(future):
    await asyncio.sleep(1)
    future.set_result('Future的结果')

async def main():
    # 创建Future对象
    future = asyncio.Future()
    
    # 创建一个任务来设置future的结果
    asyncio.create_task(set_future_result(future))
    
    # 等待future完成
    result = await future
    print('获取到结果:', result)

if __name__ == '__main__':
    asyncio.run(main())

### 2.2 Task对象
Task是Future的子类，用于包装和管理协程。当一个协程被包装成Task后，它会被自动调度执行。

In [ ]:
async def long_operation():
    print('开始长时间操作')
    await asyncio.sleep(2)
    print('长时间操作完成')
    return '操作结果'

async def main():
    # 创建task
    task = asyncio.create_task(long_operation())
    
    # 等待task完成
    print('等待任务完成...')
    result = await task
    print('获取到任务结果:', result)

if __name__ == '__main__':
    asyncio.run(main())

## 3. 异步操作

### 3.1 异步IO操作
以下是一个使用asyncio实现的异步文件读写示例：

In [ ]:
import aiofiles

async def async_file_io():
    try:
        # 异步写文件
        async with aiofiles.open('test.txt', 'w') as f:
            await f.write('Hello, Async IO!')
        
        # 异步读文件
        async with aiofiles.open('test.txt', 'r') as f:
            content = await f.read()
            print('读取到的内容:', content)
    except ImportError:
        print('请先安装aiofiles: pip install aiofiles')

if __name__ == '__main__':
    asyncio.run(async_file_io())

### 3.2 并发控制
使用Semaphore来控制并发量，避免同时运行太多协程：

In [ ]:
async def worker(name, sem):
    async with sem:
        print(f'{name} 开始工作')
        await asyncio.sleep(1)
        print(f'{name} 完成工作')

async def main():
    # 限制最大并发数为3
    sem = asyncio.Semaphore(3)
    
    # 创建10个任务
    tasks = [worker(f'worker_{i}', sem) for i in range(10)]
    await asyncio.gather(*tasks)

if __name__ == '__main__':
    asyncio.run(main())

## 4. 高级特性

### 4.1 异步上下文管理器
创建支持异步上下文管理的类：

In [ ]:
class AsyncResource:
    async def __aenter__(self):
        print('获取资源')
        await asyncio.sleep(1)  # 模拟资源获取
        return self
    
    async def __aexit__(self, exc_type, exc_val, exc_tb):
        print('释放资源')
        await asyncio.sleep(0.5)  # 模拟资源释放

async def main():
    async with AsyncResource() as resource:
        print('使用资源中...')
        await asyncio.sleep(1)

if __name__ == '__main__':
    asyncio.run(main())

## 5. 实践应用

### 5.1 高并发网络爬虫
使用aiohttp实现的异步网络请求示例：

In [ ]:
import aiohttp

async def fetch_url(session, url):
    try:
        async with session.get(url) as response:
            return await response.text()
    except Exception as e:
        print(f'获取 {url} 失败: {e}')
        return None

async def main():
    urls = [
        'http://example.com',
        'http://example.org',
        'http://example.net'
    ]
    
    try:
        async with aiohttp.ClientSession() as session:
            tasks = [fetch_url(session, url) for url in urls]
            results = await asyncio.gather(*tasks)
            for url, html in zip(urls, results):
                if html:
                    print(f'{url}: 获取成功，长度 {len(html)}')
    except ImportError:
        print('请先安装aiohttp: pip install aiohttp')

if __name__ == '__main__':
    asyncio.run(main())

## 6. 最佳实践

### 6.1 异常处理
在异步编程中正确处理异常的示例：

In [ ]:
async def might_fail():
    await asyncio.sleep(1)
    raise ValueError('操作失败')

async def handle_error():
    try:
        await might_fail()
    except ValueError as e:
        print(f'捕获到异常: {e}')
    finally:
        print('清理资源...')

if __name__ == '__main__':
    asyncio.run(handle_error())

## 7. 注意事项

1. **避免同步阻塞**
   - 使用异步库替代同步库
   - CPU密集型任务使用线程池

2. **资源管理**
   - 正确关闭连接和会话
   - 使用异步上下文管理器

3. **性能考虑**
   - 合理控制并发量
   - 适当使用缓存
   - 注意内存管理

以下是一个结合这些注意事项的示例：

In [ ]:
from concurrent.futures import ThreadPoolExecutor
import functools

# CPU密集型任务
def cpu_bound_task(x):
    return sum(i * i for i in range(x))

async def main():
    print('演示如何处理CPU密集型任务...')
    
    # 创建线程池
    loop = asyncio.get_event_loop()
    with ThreadPoolExecutor() as pool:
        # 在线程池中运行CPU密集型任务
        nums = [5000000, 4000000, 3000000]
        tasks = []
        for n in nums:
            # 将同步函数包装成异步任务
            task = loop.run_in_executor(pool, cpu_bound_task, n)
            tasks.append(task)
        
        # 并发执行所有任务
        results = await asyncio.gather(*tasks)
        print('计算结果:', results)

if __name__ == '__main__':
    asyncio.run(main())